# Interactive plotting with plotly

Now we'll introduce one of Python's popular plotting libraries: plotly (documented [here](https://plot.ly/python/)). Plotly's strength is in easily creating interactive plots to view in a web browser. But it's not just a library: [plot.ly](https://plot.ly) is also a website where you can find and share plots.

By default plotly runs online, meaning it saves your plot to your online plotly account. However for simplicity we'll be using it offline, just in this notebook.

First we'll catch up to where we left off in the previous notebook: filter our WFS query to Sydney 2017 and read it to a dataframe called `listings` .

In [ ]:
import pandas
wfs_query = 'https://citydata.be.unsw.edu.au/geoserver/wfs?srsName=EPSG%3A4326&typename=geonode%3AInsideAirbnb_44_2015_17&outputFormat=csv&version=1.0.0&service=WFS&request=GetFeature'
maxFeatures_param = '&maxFeatures=10'
PropertyName_param = '&PropertyName=City,ScrapeDate,PropertyID,HostID,Latitude,Longitude,Price'
cql_filter_param = "&cql_filter=City='Sydney'+AND+ScrapeDate='2017-04-03T00:00:00'"

url = wfs_query + PropertyName_param + cql_filter_param #  + maxFeatures_param
listings = pandas.read_csv(url)

listings.head()

Before we can use plotly  we need to import it. To save typing later though, we'll import just the bits we need and assign a short alias to `graph_objs`.

*TODO: explain graph_objs, notebook mode and iplot.*

In [ ]:
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot

init_notebook_mode() # Plot data in the notebook not to a file

For our first plot let's just use latitude by longitude as a scatter plot to produce a simple map.

*TODO: explain trace, layout, figure.*

In [ ]:
trace = go.Scatter(
                    x=listings['Longitude'], y=listings['Latitude'], # Data
                    mode='markers', name='map' # Additional options
                   )

layout = go.Layout(title='Simple Map of Sydney Airbnb Listings 2017',
                   plot_bgcolor='rgb(230, 230,230)')

fig = go.Figure(data=[trace], layout=layout)

iplot(fig)

*TODO: note the interactive widgets and their use*

Sydney is a coastal city. Where are most of the Airbnb listings?

Generally, houses are more expensive nearer to the coast.

Does the same apply to Airbnb? Let's plot price against longitude to find out.

**Copy the code above into the cell below, and change it to plot Price instead of Latitude. Change the title too.**

*TODO use https://plot.ly/python/line-and-scatter/#scatter-with-a-color-dimension to make a heatmap by price from the above

## Aggregation

Let's look at the Airbnb hosts. How many listings does each host manage? We need to aggregate the data by HostID and plot the number of listings for each host. To do this we must construct the data 'manually' rather than using the Scatter function.

In [ ]:
trace = [{
  'type': 'scatter',
  'x': listings['HostID'],
  'y': listings['PropertyID'],
  'mode': 'markers',
  'transforms': [{
    'type': 'aggregate',
    'groups': listings['HostID'],
    'aggregations': [{'target': 'y', 'func': 'count', 'enabled': True}]
  }]
}]

layout = go.Layout(title='Sydney Airbnb Listings Per Host 2017',
                   plot_bgcolor='rgb(230, 230,230)')

iplot({'data': trace, 'layout': layout}, validate=False)

Some Airbnb hosts manage over a hundred properties!

How much are these portfolios worth? Let's look at potential nightly income, i.e. the sum of the nightly rates for all properties hosted by each host.

In [ ]:
host_by_total_price = [dict(
  type = 'scatter',
  x = listings['HostID'],
  y = listings['Price'],
  mode = 'markers',
  transforms = [dict(
    type = 'aggregate',
    groups = listings['HostID'],
    aggregations = [dict(
        target = 'y', func = 'sum', enabled = True),
    ]
  )]
)]

iplot({'data': host_by_total_price}, validate=False)


Wow some of those portfolios are huge!

*TODO: bar chart ...of?*

*TODO: pie chart ...of entire home vs room?*

